In [2]:

from bs4 import BeautifulSoup
import sqlite3
import requests
import pandas as pd
from random import randint
import time
from time import sleep
from typing import List
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score

In [225]:
def sleepms(milliseconds: int):
    seconds = 0.001 * milliseconds
    sleep(seconds)
    
def random_wait():
    start = time.time()
    sleepms(randint(1, 350))

def getFakeUser():
    return {"User-Agent":"PostmanRuntime/7.29.0"}


def getRetHeaders():
    url='https://www.ret.co.il/m/91935/specs'
    page = requests.get(url, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    product=soup.find('div',class_='single-product-tab')
    data=product.findAll('ul','list-group list-group-horizontal')
    myheaders= ['מחיר מינימלי','מחיר מקסימלי']
    for alldata in data:
        myheaders.append(alldata.find('span',class_='text-dark font-weight-bold').text.strip())
    return myheaders



def getUrl(url):
    page = requests.get(url, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    MainClass=soup.find('ul', class_='nav nav-pills mb-2 justify-content-around justify-content-md-start col-xxl-10 offset-xxl-1 col-md-12 offset-md-0')
    ref=[]
    data=MainClass.find('a')
    technicalDataPage='https://www.ret.co.il'+data.get('href')
    return technicalDataPage

def getRetData(url):
    technicalDataPage=getUrl(url)
    page = requests.get(technicalDataPage, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    product=soup.find('div',class_='single-product-tab')
    dataLink=product.findAll('ul',class_='list-group list-group-horizontal')
    MinMaxP=MinMaxRetPrices(url)
    mydata=[MinMaxP[0],MinMaxP[1]]
    myheaders=[]
    for alldata in dataLink:
        # print(alldata.find('li',class_='list-group-item col-8 rounded-0 text-600').text.strip())
        try:
            mydata.append(alldata.find('li',class_='list-group-item col-8 rounded-0 text-600').text.strip())
        except:
             mydata.append('None')
    return mydata

def MinMaxRetPrices(url):
    page = requests.get(url, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    MainClass=soup.find('div',class_='')
    MainClass=soup.find('div',class_='card-body p-0 p-sm-3')
    price=MainClass.find('div',class_='price')
    if(price==None):
        return [None,None]
    else:
        price=price.text.strip()
        price=price.replace(',','')
        prices=price.split()
        MinPrice=prices[0]
        MaxPrice=prices[3]
        priceArray=MinPrice,MaxPrice
        return priceArray


In [209]:
def getRetToDataFrame(phonesURL):
    hrefLinks=[]
    allRetData=[]
    page = requests.get(phonesURL, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    MainClass=soup.findAll('a')
    for href in MainClass:
        if('/m/' in href.get('href')):
            hrefLinks.append('https://www.ret.co.il' + href.get('href'))
    for hrefLink in hrefLinks:
        allRetData.append(getRetData(hrefLink))
    return allRetData


In [210]:
def mainFunc(page_num_ret):
    head=getRetHeaders()
    url=f"https://www.ret.co.il/shop/cellphone?pageindex={page_num_ret}"
    data=getRetToDataFrame(url)
    df=pd.DataFrame(data,columns=head)
    return df

In [226]:
df_ret=mainFunc(1)
for i in range(2,10):
    df_ret=df_ret.append(mainFunc(i))
    random_wait()

C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\3648987791.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\3648987791.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\3648987791.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\3648987791.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\3648987791.

In [212]:
def get_headers():
    url="https://www.zap.co.il/compmodels.aspx?modelid=1142728"
    page = requests.get(url, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    All_headers=['מחיר מינימלי','מחיר מקסימלי']
    headers=soup.find_all('div',class_="detailsRow")
    for header in headers:
        All_headers.append(header.find('div',class_="detailsRowTitletxt").text.strip())
    return All_headers


In [213]:
def get_properites_from_link(url):
    All_data=[]
    All_headers=[]
    page = requests.get(url, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    headers=soup.find_all('div',class_="detailsRow")
    titles = ['מחיר מינימלי','מחיר מקסימלי']
    prices = soup.find('div','PricesTxt').findAll('span') 
    if len(prices) == 2:
        All_data = [ prices[1].text, prices[0].text ]
    elif len(prices) == 1: 
        All_data = [ prices[0].text, prices[0].text ]
    else:
        All_data = [ None,None ]
    for header in headers:
        All_headers.append(header.find('div',class_="detailsRowTitletxt").text.strip())
        All_data.append(header.find('div',class_="detailsRowTxt").text.strip())
    return All_data

In [214]:
def get_data_from_zap(phonesURL):
    hrefLinks=[]
    all_zap_Data=[]
    Main_Url="https://www.zap.co.il"
    page = requests.get(phonesURL, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    links=soup.find_all('div',class_="MoreInfo")
    for link in links:
        hrefLinks.append(Main_Url+(link.find('a')['href']))
    for link in hrefLinks:
        all_zap_Data.append(get_properites_from_link(link))
    return all_zap_Data
    

In [215]:
def get_details_from_current_page(page_num_zap):
    url="https://www.zap.co.il/models.aspx?sog=e-cellphone&pageinfo={page_num_zap}"
    data=get_data_from_zap(url)
    df=pd.DataFrame(data,columns=get_headers())
    return df



In [231]:

def change_CSV_File_Headers(df):
    df.rename(columns={"פוקוס אוטומטי": "Autofocus"}, inplace=True)
    df.rename(columns={"פלאש": "Flash"}, inplace=True)
    df.rename(columns={"מצלמה": "Camera"}, inplace=True)
    df.rename(columns={"מפעיל": "operator"}, inplace=True)
    df.rename(columns={"דור": "Generation"}, inplace=True)
    df.rename(columns={"סוג רשת": "Network kind"}, inplace=True)
    df.rename(columns={"כרטיס SIM": "SIM"},inplace=True)
    df.rename(columns={"רדיו FM": "FM radio"}, inplace=True)
    df.rename(columns={"גובה": "height"}, inplace=True)
    df.rename(columns={"רוחב": "width"}, inplace=True)
    df.rename(columns={"עובי": "thickness"}, inplace=True)
    df.rename(columns={"משקל": "wheight"}, inplace=True)
    df.rename(columns={"קיבולת סוללה": "Battery capacity"}, inplace=True)
    df.rename(columns={"סוללה נשלפת": "Removable battery"}, inplace=True)
    df.rename(columns={"רמקול חיצוני": "External speaker"}, inplace=True)
    df.rename(columns={"חיבור 3.5": "connection 3.5"},inplace=True)
    df.rename(columns={"מחיר מינימלי": "Minprice"}, inplace=True)
    df.rename(columns={"מחיר מקסימלי": "Maxprice"}, inplace=True)
    df.rename(columns={"יצרן": "Brand"}, inplace=True)
    df.rename(columns={"שנת הכרזה": "Declaration year"}, inplace=True)
    df.rename(columns={"מערכת הפעלה": "Operating System"}, inplace=True)
    df.rename(columns={"סדרה": "series"}, inplace=True)
    df.rename(columns={"מבנה המכשיר": "device structure"},inplace=True)
    df.rename(columns={"ממשק הפעלה": "operating interfaceo"}, inplace=True)
    df.rename(columns={"נפח אחסון פנימי": "Internal storage volume"}, inplace=True)
    df.rename(columns={"ערכת שבבים": "chipset"}, inplace=True)
    df.rename(columns={"צבעים": "colors"}, inplace=True)
    df.rename(columns={"טלפון כשר": "kosher phone"}, inplace=True)
    df.rename(columns={"רמת קרינה (SAR)": "radiation level"}, inplace=True)
    df.rename(columns={"טלפון למבוגרים": "Phone for adults"}, inplace=True)
    df.rename(columns={"מאיץ גרפי": "Graphics accelerator"}, inplace=True)
    df.rename(columns={"מהירות מעבד": "CPU speed"},inplace=True)
    df.rename(columns={"עמידות במים": "Water resistance"}, inplace=True)
    df.rename(columns={"קורא טביעת אצבע": "Fingerprint reader"},inplace=True)
    df.rename(columns={"גודל מסך": "screen size"}, inplace=True)
    df.rename(columns={"סוג מסך": "screen type"}, inplace=True)
    df.rename(columns={"קצב רענון": "refresh rate"}, inplace=True)
    df.rename(columns={"צפיפות פיקסלים": "Pixel density"}, inplace=True)
    df.rename(columns={"מפתח צמצם": "Aperture key"}, inplace=True)
    df.rename(columns={"מעבד": "Processor"}, inplace=True)
    df.rename(columns={"רזולוציה": "resolution"},inplace=True)
    df.rename(columns={"כרטיס זיכרון":"Memory Card"}, inplace=True)
    df.rename(columns={"זיכרון RAM": "RAM memory"}, inplace=True)
    df.rename(columns={"וידאו": "video"}, inplace=True)
    df.rename(columns={"קיבולת הסוללה": "battery capacity"}, inplace=True)
    df.rename(columns={"מספר ליבות": "number of cores"}, inplace=True)
    df.rename(columns={"מצלמה קדמית": "front camera"}, inplace=True)
    df.rename(columns={"טעינה אלחוטית": "wireless charging"}, inplace=True)
    df.to_csv("PhonesDF.csv",index=False)


In [217]:
df_zap=get_details_from_current_page(1)
for i in range(2,21):
    df_zap=df_zap.append(get_details_from_current_page(i))
    random_wait()

C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\2310696407.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_details_from_current_page(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\2310696407.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_details_from_current_page(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\2310696407.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_details_from_current_page(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\2310696407.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_detail

In [3]:
df=df_zap.append(df_ret)
df.to_csv('PhonesDF.csv',header=True)
df=pd.read_csv('PhonesDF.csv')
change_CSV_File_Headers(df)
df=pd.read_csv('PhonesDF.csv')
df


NameError: name 'df_zap' is not defined

In [8]:
df=pd.read_csv('PhonesDF.csv')
df=df.drop_duplicates()
df

,Unnamed: 0,Minprice,Maxprice,Brand,Operating System,Declaration year,device structure,series,Smartphone,radiation level,...,External speaker,FM radio,height,width,thickness,wheight,battery capacity,Removable battery,wireless charging,Dual-SIM
0,0,"3,458","4,649",Samsung,Android,2022,פאבלט,Galaxy S22,סמארטפון,1.05 W/kg,...,זוג רמקולים,יעודכן בקרוב,"163.3 מ""מ","77.9 מ""מ","8.9 מ""מ",228 גרם,"5,000 mAh",לא,כולל,NaN
1,1,"2,774","3,499",Apple,iOS,2021,פאבלט,iPhone 13,סמארטפון,יעודכן בקרוב,...,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.7 מ""מ",174 גרם,לא זמין,לא,כולל,NaN
2,2,"5,795","6,699",Apple,iOS,2022,פאבלט,iPhone 14 Pro Max,סמארטפון,לא זמין,...,זוג רמקולים,ללא רדיו,"160.7 מ""מ","77.6 מ""מ","7.9 מ""מ",240 גרם,לא זמין,לא,כולל,NaN
3,3,"1,510","1,949",Samsung,Android,2022,פאבלט,Galaxy A73,סמארטפון,1.28 W/kg,...,יחיד,יעודכן בקרוב,"163.7 מ""מ","76.1 מ""מ","7.6 מ""מ",181 גרם,"5,000 mAh",לא,ללא,NaN
4,4,"3,193","3,965",Apple,iOS,2022,פאבלט,iPhone 14,סמארטפון,לא זמין,...,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.8 מ""מ",172 גרם,לא זמין,לא,כולל,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,15,1899,1899,Samsung,Android,None,פאבלט,None,סמארטפון,None,...,יחיד,None,"163.7 מ""מ","76.1 מ""מ","7.6 מ""מ",181 גרם,"5,000 mAh",לא,None,None
880,16,1180,1180,Samsung,Android,2021,פאבלט,Galaxy A52,סמארטפון,1.05 W/kg,...,זוג רמקולים,כולל רדיו,"159.9 מ""מ","75.1 מ""מ","8.4 מ""מ",189 גרם,"4,500 mAh",לא,None,None
881,17,5949,5949,Samsung,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
882,18,1799,1449,Xiaomi,Android,None,פאבלט,None,סמארטפון,None,...,זוג רמקולים,None,"164.2 מ""מ","76.1 מ""מ","8.1 מ""מ",202 גרם,"5,000 mAh",לא,ללא,כולל


In [21]:
df2=pd.read_csv('PhonesDF.csv')
df2=df2[df2.duplicated()]
df2.dropna(axis=0, how='all')# if all the values are None remove this row
df2.to_csv('SmallPhonesDF.csv')
df2=pd.read_csv('SmallPhonesDF.csv')
df2

,Unnamed: 0.1,Unnamed: 0,Minprice,Maxprice,Brand,Operating System,Declaration year,device structure,series,Smartphone,...,External speaker,FM radio,height,width,thickness,wheight,battery capacity,Removable battery,wireless charging,Dual-SIM
0,24,0,"3,458","4,649",Samsung,Android,2022,פאבלט,Galaxy S22,סמארטפון,...,זוג רמקולים,יעודכן בקרוב,"163.3 מ""מ","77.9 מ""מ","8.9 מ""מ",228 גרם,"5,000 mAh",לא,כולל,NaN
1,25,1,"2,774","3,499",Apple,iOS,2021,פאבלט,iPhone 13,סמארטפון,...,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.7 מ""מ",174 גרם,לא זמין,לא,כולל,NaN
2,26,2,"5,795","6,699",Apple,iOS,2022,פאבלט,iPhone 14 Pro Max,סמארטפון,...,זוג רמקולים,ללא רדיו,"160.7 מ""מ","77.6 מ""מ","7.9 מ""מ",240 גרם,לא זמין,לא,כולל,NaN
3,27,3,"1,510","1,949",Samsung,Android,2022,פאבלט,Galaxy A73,סמארטפון,...,יחיד,יעודכן בקרוב,"163.7 מ""מ","76.1 מ""מ","7.6 מ""מ",181 גרם,"5,000 mAh",לא,ללא,NaN
4,28,4,"3,193","3,965",Apple,iOS,2022,פאבלט,iPhone 14,סמארטפון,...,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.8 מ""מ",172 גרם,לא זמין,לא,כולל,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,475,19,"2,396","2,855",Xiaomi,Android,2022,פאבלט,12T Pro,סמארטפון,...,זוג רמקולים,ללא רדיו,"163.1 מ""מ","75.9 מ""מ","8.6 מ""מ",205 גרם,"5,000 mAh",לא,לא זמין,NaN
447,476,20,"3,870","4,799",Apple,iOS,2021,פאבלט,iPhone 13,סמארטפון,...,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.7 מ""מ",204 גרם,לא זמין,לא,כולל,NaN
448,477,21,"1,988","2,690",Apple,iOS,2019,פאבלט,iPhone 11,סמארטפון,...,זוג רמקולים,ללא רדיו,"150.9 מ""מ","75.7 מ""מ","8.3 מ""מ",194 גרם,"3,110 mAh",לא,כולל,NaN
449,478,22,"1,288","1,949",Google,Android,2022,פאבלט,Pixel 6a,סמארטפון,...,זוג רמקולים,ללא רדיו,"152.2 מ""מ","71.8 מ""מ","8.9 מ""מ",178 גרם,"4,410 mAh",לא,ללא,NaN


In [ ]:
df2=pd.read_csv('SmallPhonesDF.csv')
df2